# Visualising the network of 360 Giving grants

## Preparing the data

This notebook takes you through the process of preparing grants data
to produce a network diagram showing relationships between funders.
In this notebook I will:

1. Fetch grants data from [GrantNav](http://grantnav.threesixtygiving.org/)
2. Transform it into a tables of links between funders and recipients
3. Export those tables to the right format

The resulting table shows which funders share recipients in common with
other funders.

The first step is to import the [pandas](https://pandas.pydata.org/) data 
analysis library. I'll be using this to fetch and transform the data.

In [1]:
import pandas as pd

### 1. Fetch grants data

First I make a variable containing a URL to a GrantNav CSV file.
This link was found by performing a search on GrantNav and then
right clicking on the "CSV" download button in the top right, 
and selecting "Copy link location" (the exact way of copying the
link may vary in different browsers).

In [2]:
grantnav_url = 'http://grantnav.threesixtygiving.org/search.csv?json_query=%7B%22aggs%22%3A+%7B%22fundingOrganization%22%3A+%7B%22terms%22%3A+%7B%22field%22%3A+%22fundingOrganization.id_and_name%22%2C+%22size%22%3A+3%7D%7D%2C+%22recipientRegionName%22%3A+%7B%22terms%22%3A+%7B%22field%22%3A+%22recipientRegionName%22%2C+%22size%22%3A+3%7D%7D%2C+%22currency%22%3A+%7B%22terms%22%3A+%7B%22field%22%3A+%22currency%22%2C+%22size%22%3A+3%7D%7D%2C+%22recipientDistrictName%22%3A+%7B%22terms%22%3A+%7B%22field%22%3A+%22recipientDistrictName%22%2C+%22size%22%3A+3%7D%7D%2C+%22recipientOrganization%22%3A+%7B%22terms%22%3A+%7B%22field%22%3A+%22recipientOrganization.id_and_name%22%2C+%22size%22%3A+3%7D%7D%7D%2C+%22query%22%3A+%7B%22bool%22%3A+%7B%22filter%22%3A+%5B%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%2C+%22must%22%3A+%7B%7D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%7B%22range%22%3A+%7B%22amountAwarded%22%3A+%7B%7D%7D%7D%2C+%22must%22%3A+%7B%7D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%7D%7D%5D%2C+%22must%22%3A+%7B%22query_string%22%3A+%7B%22default_field%22%3A+%22_all%22%2C+%22query%22%3A+%22awardDate%3A%5B2016-01-01+TO+2017-12-31%5D%22%7D%7D%7D%7D%2C+%22extra_context%22%3A+%7B%22amountAwardedFixed_facet_size%22%3A+3%2C+%22awardYear_facet_size%22%3A+3%7D%2C+%22sort%22%3A+%7B%22_score%22%3A+%7B%22order%22%3A+%22desc%22%7D%7D%7D'

Make a list of the columns I want to use in the data - only a small number are relevant for this exercise.

In [3]:
columns = [
    "Identifier", "Currency", "Amount Awarded", "Award Date",
    "Recipient Org:Identifier", "Recipient Org:Name", 
    "Recipient Org:Charity Number", "Recipient Org:Company Number",
    "Funding Org:Identifier", "Funding Org:Name"
]

Then I use pandas to fetch the data. The `read_csv` method accepts an
`index_col` parameter which tells it which column to use as an index, while
passing our columns to `usecols` means only those columns will be returned.

This can take a little while to run as it downloads a large amount of data.

In [4]:
grants = pd.read_csv(grantnav_url, index_col='Identifier', usecols=columns)

I've added a dummy `Grants` variable with a value of 1 for each row. This
will help later when I want to count the number of grants.

In [5]:
grants.loc[:, "Grants"] = 1

Some funders put their name with spaces at the end, so I need to get rid of them.

In [6]:
grants.loc[:, "Funding Org:Name"] = grants["Funding Org:Name"].str.strip()

Turn the date field into a date format.

In [7]:
grants.loc[:, "Award Date"] = grants["Award Date"].astype('datetime64[ns]')

Let's take a look at the resulting data - first see how many rows there are:

In [8]:
len(grants)

60926

Then preview the list itself

In [9]:
grants

,Currency,Amount Awarded,Award Date,Recipient Org:Identifier,Recipient Org:Name,Recipient Org:Charity Number,Recipient Org:Company Number,Funding Org:Identifier,Funding Org:Name,Grants
Identifier,,,,,,,,,,
360G-SomersetCF-A397976,GBP,1470.00,2016-03-23,GB-CHC-1141319,Young People Frome,1141319,NaN,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A397978,GBP,2000.00,2016-04-04,GB-COH-09284683,GoCreate Taunton CIC,NaN,09284683,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A401718,GBP,2500.00,2016-06-24,360G-SomersetCF-ACC351246,St Francis Youth Club,NaN,NaN,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A404393,GBP,500.00,2016-06-30,360G-SomersetCF-ACC763854,Yeovil & Sherborne Hockey Club,NaN,NaN,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A404576,GBP,500.00,2016-06-07,GB-CHC-270392,Ashill Village Hall Committee,270392,NaN,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A405480,GBP,9962.00,2016-06-14,GB-COH-04983733,Tone Leisure (South West) Ltd,1110756,04983733,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A405664,GBP,11000.00,2016-06-14,GB-CHC-305633,"Blackdown District Scouts, Tangier Scout & Gui...",305633,NaN,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A406072,GBP,500.00,2016-06-30,360G-SomersetCF-ACC766126,Burnham United Junior Football Club,NaN,NaN,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A406345,GBP,500.00,2016-06-01,360G-SomersetCF-ACC350213,Good Fellowship Club,NaN,NaN,GB-COH-04530979,Somerset Community Foundation,1


Save the data to a pickle so we can use it later

In [10]:
grants.to_pickle('grants.pkl')

### 2. Transform into a new format

I might want to filter the data before plotting it, so it's useful to copy to
a new variable to keep the original `grants` variable as is. 

In this case I've filtered to only include grants in 2017.

In [11]:
to_use = grants[grants["Award Date"].dt.year == 2017]
print("Using {:,.0f} grants".format(len(to_use)))

Using 31,290 grants


I create a table showing links between funders and recipients.
This has one row per funding relationship, from Funder > Recipient. 
I've added some columns showing the number of grants made by each 
funder to the recipient and the value of those grants.

To do this I use the `groupby` pandas function to group, and then
`sum` to pick up the amount and number of grants. I've reset the index
to make it easier to use the table later on.

In [12]:
links = to_use.groupby(
    ["Recipient Org:Identifier", "Funding Org:Identifier"]
).sum()[["Amount Awarded", "Grants"]].reset_index()

I only want to use links including recipients that have received
grants from more than one funder, to limit the size of the network
diagram. 

To do this I've used the `value_counts()` function to get a list
of unique recipients and how many relationships they occur in.

In [13]:
link_recipients = links["Recipient Org:Identifier"].value_counts()
print("{:,.0f} unique recipients".format(len(link_recipients)))

24,976 unique recipients


Then I filter this list to include only those links where the
recipient appears more than once.

In [14]:
links = links[links["Recipient Org:Identifier"].isin(link_recipients[link_recipients>1].index)]

We also add in the names of the funders to help with the data.

In [15]:
links = links.join(grants.groupby("Funding Org:Identifier").first()["Funding Org:Name"], on="Funding Org:Identifier")

In [16]:
print("{:,.0f} unique recipients with more than one funder in the period".format(len(links["Recipient Org:Identifier"].value_counts())))

2,316 unique recipients with more than one funder in the period


In [17]:
links

,Recipient Org:Identifier,Funding Org:Identifier,Amount Awarded,Grants,Funding Org:Name
7985,360G-trafford-theatre_of_the_senses,GB-COH-04831118,467.00,1,Trafford Housing Trust Social Investment
7986,360G-trafford-theatre_of_the_senses,GB-LAE-TRF,1720.00,1,Trafford Metropolitan Borough Council
8067,GB-CHC-1000011,360G-blf,8348.00,1,The Big Lottery Fund
8068,GB-CHC-1000011,GB-CHC-226446,39296.00,2,Seafarers UK
8075,GB-CHC-1000340,360G-blf,464600.00,1,The Big Lottery Fund
8076,GB-CHC-1000340,GB-CHC-230260,10000.00,1,Garfield Weston Foundation
8077,GB-CHC-1000351,360G-blf,10000.00,1,The Big Lottery Fund
8078,GB-CHC-1000351,GB-COH-IP00525R,3162.39,1,Co-operative Group
8092,GB-CHC-1000714,360G-blf,9920.00,1,The Big Lottery Fund
8093,GB-CHC-1000714,GB-CHC-230260,5000.00,1,Garfield Weston Foundation


### Create Chord output

The next step is to turn the links into a table with the number 
of recipients that pairs of funders have in common.

First I get a list of the funders in the dataset.

In [18]:
funders = links["Funding Org:Name"].unique()

Then go through each funder and get a list of the recipients of the funder,
and which other funders have funded them.

In [19]:
funder_rels = {}
# go through all the funders
for f in funders:
    
    # a list of the recipients of this funder
    recipients = links.loc[
                links["Funding Org:Name"]==f, 
                "Recipient Org:Identifier"].unique()
    
    # get a list of all the other funders that have funded those recipients
    funder_rels[f] = pd.DataFrame(links.loc[
        links["Recipient Org:Identifier"].isin(recipients) & (links["Funding Org:Name"]!=f),
        "Funding Org:Name"
    ].value_counts().rename("grants"))

Turn into a list of funder > funder relationships.

In [20]:
funder_rels = pd.concat(funder_rels, names=["Funder from", "Funder to"])

Save to a CSV file.

In [21]:
funder_rels.to_csv("chord.csv")

This CSV file is in the right format to use in a [Flourish chord diagram](https://app.flourish.studio/@flourish/chord-diagram).

## Create stats

We want to find out:

- for each funder, what proportion of recipients are found in other funders
- what the average grant size is for the funder

In [31]:
funder_count = to_use.groupby(
    ["Recipient Org:Identifier", "Funding Org:Name"]
).sum()[
    ["Amount Awarded", "Grants"]
].reset_index().join(
    link_recipients.rename('funder_count'), on='Recipient Org:Identifier'
)
funder_count

,Recipient Org:Identifier,Funding Org:Name,Amount Awarded,Grants,funder_count
0,360G-ArcadiaFund:ORG-University-of-Hamburg,ARCADIA,2000000.00,1,1
1,360G-BarnetCouncil-ORG:1st-3rd-New-Barnet-Scou...,London Borough of Barnet,5000.00,1,1
2,360G-BarnetCouncil-ORG:Barnet-Bowls-Club,London Borough of Barnet,5000.00,1,1
3,360G-BarnetCouncil-ORG:Finchley-Horticultural-...,London Borough of Barnet,4950.00,1,1
4,360G-BarnetCouncil-ORG:Mill-Hill-Neighbourhood...,London Borough of Barnet,750.00,1,1
5,360G-BarnetCouncil-ORG:Stonegrove-Estates-Yout...,London Borough of Barnet,2405.00,1,1
6,360G-BarnetCouncil-ORG:The-Hope-Of-Childs-Hill,London Borough of Barnet,5000.00,2,1
7,360G-BirminghamCC-acocks_green_nhood_forum,Birmingham City Council,800.00,1,1
8,360G-BirminghamCC-bham_neighbourhood_forum,Birmingham City Council,610.00,2,1
9,360G-BirminghamCC-boldmere_neighbourhood_forum,Birmingham City Council,700.00,1,1


In [55]:
link_count = pd.crosstab(
    funder_count['Funding Org:Name'],
    funder_count['funder_count'],
    margins=True
).sort_values(
    by='All', ascending=False
).drop('All')
pd.DataFrame((1-(link_count[1] / link_count['All'])).multiply(100).round(1))

,0
Funding Org:Name,
The Big Lottery Fund,12.2
Co-operative Group,14.7
Garfield Weston Foundation,38.5
Sport England,6.7
Quartet Community Foundation,16.9
The Robertson Trust,38.5
Heart Of England Community Foundation,20.0
The Tudor Trust,48.0
Lloyds Bank Foundation for England and Wales,58.6


In [52]:
(pd.crosstab(
    funder_count['Funding Org:Name'],
    funder_count['funder_count'],
    values=funder_count['Amount Awarded'],
    aggfunc='sum',
    margins=True,
) / pd.crosstab(
    funder_count['Funding Org:Name'],
    funder_count['funder_count'],
    values=funder_count['Grants'],
    aggfunc='sum',
    margins=True,
)).fillna(0).round(0)

funder_count,1,2,3,4,5,6,All
Funding Org:Name,,,,,,,
A B Charitable Trust,13000.0,13400.0,12344.0,14667.0,15000.0,0.0,13147.0
ARCADIA,2322464.0,16581364.0,0.0,0.0,0.0,0.0,3463176.0
Barrow Cadbury Trust,44969.0,34350.0,44762.0,91500.0,0.0,0.0,43859.0
Birmingham City Council,13472.0,10367.0,0.0,22699.0,0.0,0.0,13285.0
"Calouste Gulbenkian Foundation, UK Branch",33627.0,0.0,0.0,0.0,0.0,0.0,33627.0
Cheshire Community Foundation,5293.0,6386.0,5295.0,7135.0,0.0,0.0,5622.0
City Bridge Trust,71616.0,102108.0,79198.0,100975.0,0.0,87900.0,84614.0
Co-operative Group,2424.0,2756.0,3601.0,2743.0,3294.0,2500.0,2506.0
Comic Relief,274863.0,90133.0,97250.0,62410.0,0.0,0.0,247159.0


In [54]:
pd.crosstab(
    funder_count['Funding Org:Name'],
    funder_count['funder_count'],
    values=funder_count['Amount Awarded'],
    aggfunc='median',
    margins=True,
).fillna(0).round(0)

funder_count,1,2,3,4,5,6,All
Funding Org:Name,,,,,,,
A B Charitable Trust,10000.0,15000.0,12500.0,20000.0,15000.0,0.0,15000.0
ARCADIA,1125000.0,16581364.0,0.0,0.0,0.0,0.0,1625000.0
Barrow Cadbury Trust,36000.0,27550.0,44762.0,91500.0,0.0,0.0,36000.0
Birmingham City Council,8659.0,6100.0,0.0,22699.0,0.0,0.0,8659.0
"Calouste Gulbenkian Foundation, UK Branch",29000.0,0.0,0.0,0.0,0.0,0.0,29000.0
Cheshire Community Foundation,4800.0,7700.0,5000.0,8954.0,0.0,0.0,5000.0
City Bridge Trust,68650.0,100000.0,91450.0,101175.0,0.0,87900.0,87750.0
Co-operative Group,2302.0,2819.0,2943.0,3353.0,4077.0,5539.0,2384.0
Comic Relief,114338.0,74934.0,89850.0,62410.0,0.0,0.0,100000.0
